In [1]:
import pandas as pd
import numpy as np

# combined data - 2 tables

In [2]:
path_tables = "C:/Users/seoul/Dropbox/00 technical/github/nkelites/data/combined data/combined data - 2 tables/"

In [3]:
filename_careers = "careers.xlsx"

In [4]:
filename_leadercareerlink = "leadercareerlink.xlsx"

In [5]:
filename_leaderjoblink = "leaderjoblink.xlsx"

In [6]:
filename_joborglink = "joborglink.xlsx"

# combined data - 1 cleaning/cleaning step 2 - career/

In [7]:
path_cleaning = "C:/Users/seoul/Dropbox/00 technical/github/nkelites/data/combined data/combined data - 1 cleaning/cleaning step 2 - career/"

In [8]:
subpath_2_1 = "2.1 career_undivided_unparsed_uncoded/"
subpath_2_2 = "2.2 career_divided_unparsed_uncoded/"
subpath_2_3 = "2.3 joborglink/"
subpath_2_4 = "2.4 orgtree/"
subpath_2_5 = "2.5 position/"
subpath_2_6 = "2.6 career_reassembled/"
subpath_2_7 = "2.7 orgtree position & rank/"

In [10]:
# 1.0 orgtree_position_rank

# constructing new file from:
## 2022 북한_기관별_인명록_북한정보포털 게재용
## orgtree table

filein_opr = "1.0 orgtree_position_rank.xlsx"
opr = pd.read_excel(path_cleaning + subpath_2_7 + filein_opr,dtype="str")
opr.shape

(162, 12)

In [11]:
# orgtree_copy

# copy of orgtree table

filein_ot = "orgtree_copy.xlsx"
ot = pd.read_excel(path_cleaning + subpath_2_7 + filein_ot,dtype="str")
ot.shape

(1742, 9)

In [26]:
# position_rank_esther - original

# original positions of 노동당, before hand-coding

filein_posrank_party_orig = "position_rank_esther.xlsx"
pr_party_orig = pd.read_excel(path_cleaning + subpath_2_7 + filein_posrank_party_orig,dtype="str")
pr_party_orig.shape

(263, 7)

In [27]:
# position_rank_esther_handcoded

# Esther's hand_ranked positions of 노동당

filein_posrank_party = "position_rank_esther_handcoded.xlsx"
pr_party = pd.read_excel(path_cleaning + subpath_2_7 + filein_posrank_party,dtype="str")
pr_party.shape

(263, 11)

In [28]:
# position_rank_jeongsu_handcoded

# Jeongsu's hand-coded positions of all orgs minus 노동당, 내각, 정무원

filein_posrank_else = "position_rank_jeongsu.xlsx"
pr_else = pd.read_excel(path_cleaning + subpath_2_7 + filein_posrank_else,dtype="str")
pr_else.shape

(842, 7)

# functions: format orgtree

In [15]:
# using this on (PI,OrgName) will ensure unique & non-null keys
# using this on a larger df will ensure unique rows and non-null keys, but not unique keys

def unique_non_null_rows(olddf):
    
    df = olddf.copy()
    
    
    ### drop duplicates
    df = df.drop_duplicates(keep="first",ignore_index=True)

    ### drop null rows
    df.dropna(how="all",axis=0,inplace=True)
    
    # drop rows with null PI
    df = df[~(df["PrimaryInstitution"].isna())]
        
    ### drop rows where PI contains stop words
    stop_words_lower = ["uncertain","current","deprecated","please_revise"]
    df = df[ ~ df["PrimaryInstitution"].str.lower().isin(stop_words_lower)]
            

    print("\nUnique Non-null Rows...")
    print("")
    print("\tNon-unique rows:",olddf.shape)
    print("\tUnique rows    :",df.shape)

    ### sort
    df = df.sort_values(["PrimaryInstitution","OrgName"])
    
    return df

In [16]:
def verify_unique_rows(df):
    
    key_columns = ["PrimaryInstitution","OrgName"]
    print("\nVerifying Unique Rows...")
    print("")
    print("\tDuplicate Rows:",df[df.duplicated(keep=False)].shape)
    print("\tDuplicate Keys:",df[df.duplicated(key_columns,keep=False)].shape)
    print("\tNull Rows     :",df[df["PrimaryInstitution"].isna() & df["OrgName"].isna()].shape)

In [17]:
def merge_results(m):
    
    print("\nMerge Results...")
    print("")
    print("\tshape     :",m.shape)
    print("\tleft_only :",m[m["_merge"]=="left_only"].shape)
    print("\tboth      :",m[m["_merge"]=="both"].shape)
    print("\tright_only:",m[m["_merge"]=="right_only"].shape)

# 0. pr_party - restore OrgName, clean file

In [34]:
# position_rank_esther_handcoded: restore OrgName from position_rank_esther
# Esther changed a few OrgNames; I will restore the original OrgName, but save Esther's changes

In [35]:
pr_party_orig.head(2)

,Unnamed: 0,InsideGov,PrimaryInstitution,OrgName,Position,IsElected,PositionRank
0,884,1,노동당,Uncertain,부부장,NaN,NaN
1,886,1,노동당,당대표자회_집행부,성원,NaN,NaN


In [30]:
pr_party.head(2)

,Unnamed: 0,InsideGov,PrimaryInstitution,OrgName,Position,IsElected,OrgLevel,PositionRank,Notes,NewOrgname,NewPosition
0,884,1,노동당,Uncertain,부부장,NaN,NaN,?,NaN,NaN,NaN
1,886,1,노동당,당대표자회_집행부,성원,NaN,NaN,NaN,"NotOrg - 대표자회는 임시 회의, '임시 당대회'",NaN,NaN


In [115]:
key_column = ["Unnamed: 0"]
orig_columns = ["Unnamed: 0","OrgName"]
pr_party2 = pr_party.merge(pr_party_orig[orig_columns],on=key_column,how="outer",indicator=True,suffixes=("","_y"))
merge_results(pr_party2)


Merge Results...

	shape     : (263, 13)
	left_only : (0, 13)
	both      : (263, 13)
	right_only: (0, 13)


In [128]:
# pr_party [Changed_OrgName, Change_Position] - create & update

In [ ]:
pr_party2["Change_OrgName"]=np.nan
pr_party2["Change_Position"]=np.nan

In [117]:
pr_party2[(pr_party2.OrgName != pr_party2.OrgName_y) & pr_party2.OrgName.notna()].shape

(8, 15)

In [118]:
pr_party2.loc[pr_party2.OrgName != pr_party2.OrgName_y,"Change_OrgName"] = pr_party2.loc[pr_party2.OrgName != pr_party2.OrgName_y,"OrgName"]

In [119]:
pr_party2[pr_party2.NewOrgname.notna()].shape

(4, 15)

In [120]:
pr_party2.loc[pr_party2.NewOrgname.notna(),"Change_OrgName"] = pr_party2.loc[pr_party2.NewOrgname.notna(),"NewOrgname"]

In [121]:
pr_party2[pr_party2.Change_OrgName.notna()].shape

(12, 15)

In [122]:
pr_party2[pr_party2.NewPosition.notna()].shape

(6, 15)

In [123]:
pr_party2.loc[pr_party2.NewPosition.notna(),"Change_Position"] = pr_party2.loc[pr_party2.NewPosition.notna(),"NewPosition"]

In [124]:
pr_party2[pr_party2.Change_Position.notna()].shape

(6, 15)

In [125]:
pr_party2.OrgName = pr_party2.OrgName_y

In [126]:
pr_party2.drop(columns=["OrgName_y","NewOrgname","NewPosition","Unnamed: 0","_merge"],inplace=True)

In [127]:
pr_party2.head(2)

,InsideGov,PrimaryInstitution,OrgName,Position,IsElected,OrgLevel,PositionRank,Notes,Change_OrgName,Change_Position
0,1,노동당,Uncertain,부부장,NaN,NaN,?,NaN,NaN,NaN
1,1,노동당,당대표자회_집행부,성원,NaN,NaN,NaN,"NotOrg - 대표자회는 임시 회의, '임시 당대회'",NaN,NaN


# 0. orgtree_position_rank: create L1_Index, L2_Index, L3_Index

In [182]:
opr.head(2)

,InsideGov,PrimaryInstitution,OrgName,PI_Index,OrgRank,P1,P2,P3,LinkToNext_PI,LinkToNext_Org,LinkToNext_Year,Notes,L1_Index,L2_Index,L3_Index
0,1,노동당,NaN,1.0.0,0,총비서,NaN,NaN,Current,Current,Current,NaN,1,0,0
1,1,노동당,당중앙위원회,1.1.0,1,NaN,NaN,"위원, 후보위원",Current,Current,Current,NaN,1,1,0


In [167]:
opr["L1_Index"]=np.nan
opr["L2_Index"]=np.nan
opr["L3_Index"]=np.nan

In [178]:
opr.loc[opr["PI_Index"].notna(),"L1_Index"] = opr.loc[opr["PI_Index"].notna()].apply(lambda x: str(x["PI_Index"]).split(".")[0],axis=1)
opr.loc[opr["PI_Index"].notna(),"L2_Index"] = opr.loc[opr["PI_Index"].notna()].apply(lambda x: str(x["PI_Index"]).split(".")[1],axis=1)
opr.loc[opr["PI_Index"].notna(),"L3_Index"] = opr.loc[opr["PI_Index"].notna()].apply(lambda x: str(x["PI_Index"]).split(".")[2],axis=1)

In [183]:
# export opr

# fileout_opr = "1.0 orgtree_position_rank.xlsx"
# opr.to_excel(path_cleaning + subpath_2_7 + fileout_opr,index=False)

# 1. orgtree_position_rank: update 노동당

In [131]:
# align opr with pr_party2

In [149]:
pr_party2.rename(columns={"PositionRank":"PositionRank_manual"},inplace=True)

In [150]:
pr_party2["PositionRank_3P"]=np.nan

In [151]:
pr_party2.head(2)

,InsideGov,PrimaryInstitution,OrgName,Position,IsElected,OrgLevel,PositionRank_manual,Notes,Change_OrgName,Change_Position,PositionRank_3P
0,1,노동당,Uncertain,부부장,NaN,NaN,?,NaN,NaN,NaN,NaN
1,1,노동당,당대표자회_집행부,성원,NaN,NaN,NaN,"NotOrg - 대표자회는 임시 회의, '임시 당대회'",NaN,NaN,NaN


In [152]:
# recursively - 

## manually update orgtree_position_rank
## open and compare with pr_party2

In [191]:
# 1.0 orgtree_position_rank

# constructing new file from:
## 2022 북한_기관별_인명록_북한정보포털 게재용
## orgtree table

filein_opr = "1.0 orgtree_position_rank.xlsx"
opr = pd.read_excel(path_cleaning + subpath_2_7 + filein_opr,dtype="str")
opr.shape

(147, 15)

In [192]:
key_columns2 = ["PrimaryInstitution","OrgName"]
pr_party3 = pr_party2.merge(opr,on=key_columns2,how="left",indicator=True,suffixes=("","_opr"))
merge_results(pr_party3)


Merge Results...

	shape     : (263, 25)
	left_only : (151, 25)
	both      : (112, 25)
	right_only: (0, 25)


In [195]:
pd.set_option('display.max_rows', None)
select_columns = ["OrgName","PI_Index","OrgRank","Position","PositionRank_manual","P1","P2","P3","Notes","Notes_opr"] 
pr_party3.loc[pr_party3["_merge"]=="left_only",select_columns]

,OrgName,PI_Index,OrgRank,Position,PositionRank_manual,P1,P2,P3,Notes,Notes_opr
0,Uncertain,NaN,NaN,부부장,?,NaN,NaN,NaN,NaN,NaN
1,당대표자회_집행부,NaN,NaN,성원,NaN,NaN,NaN,NaN,"NotOrg - 대표자회는 임시 회의, '임시 당대회'",NaN
21,당중앙위원회_38호실,NaN,NaN,실장,1,NaN,NaN,NaN,NaN,NaN
31,당중앙위원회_강원도당위원회,NaN,NaN,비서,2,NaN,NaN,NaN,NaN,NaN
32,당중앙위원회_강원도당위원회,NaN,NaN,책임비서,1,NaN,NaN,NaN,NaN,NaN
33,당중앙위원회_강원도당위원회_문천군당위원회,NaN,NaN,책임비서,1,NaN,NaN,NaN,NaN,NaN
34,당중앙위원회_강원도당위원회_문천시당위원회,NaN,NaN,책임비서,1,NaN,NaN,NaN,NaN,NaN
35,당중앙위원회_강원도당위원회_원산시당위원회,NaN,NaN,책임비서,1,NaN,NaN,NaN,NaN,NaN
36,당중앙위원회_건설운수부,NaN,NaN,부장,1,NaN,NaN,NaN,NaN,NaN
37,당중앙위원회_건설운수사업부,NaN,NaN,부장,1,NaN,NaN,NaN,NaN,NaN


In [ ]:
# once aligned, copy pr_party2 hand-coded PositionRank into opr